STAGE 1: Calculating Df and kf

In [ ]:
# FIRST: IMPORT AND READ THE DATA
from pathlib import Path
import numpy as np
from DLCA_Reader import DLCA

# The folder with all .h5 and .xmf files
data_dir = Path("/run/media/pouxa/USB-KEY JMC/python-analysis/output_dir/")

# Read all data
Spheres, Aggregates = DLCA(data_dir).read()

In [ ]:
# SECOND: CALCULATE THE INPUTS FOR OBTAINING D_f AND k_f
# Create a new column in Aggregates that contains the giration diameter of all aggregates
Aggregates["Dg"] = 2 * Aggregates["Rg"]
Spheres["Diameter"] = 2 * Spheres["Radius"]

# Create a new column in Aggregates that contains the mean diameter the spheres of each aggregates
# This also means that we need to compute
# the mean of the diameter of all the spheres that shares the same time and the same label
# Reminder : label is the unique number given to an aggregate to recognize it
Aggregates["MeanDp"] = Spheres.groupby(by=["Time", "Label"])["Diameter"].mean()

# Compute Dg over Dp
Aggregates["DgOverDp"] = Aggregates["Dg"] / Aggregates["MeanDp"]

# Export Aggregates and pheres in .csv files
# Aggregates.to_csv("Aggregates_1.csv", header=True)
# Spheres.to_csv("Spheres_1.csv", header=True)

In [ ]:
from math import pi
from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

T_g = 1700
lambda_g = 6.84805e-08
mu_g = (18.203e-6) * (293.15 + 110) / (T_g + 110) * (T_g / 293.15) ** (1.5)
k_B = 1.38066e-23

# FUNCTION TO OBTAIN THE CUNNIGHAM, Cc
def GET_Cc(R):
    A1 = 1.142
    A2 = 0.558
    A3 = 0.999
    Kn = lambda_g / R
    return 1 + A1 * Kn + A2 * Kn * np.exp(-A3 / Kn)


# FUNCTION TO OBTAIN THE MOBILITY DIAMETER, Dm, from f_agg
def GET_Dm(Cc_as, fa):
    return fa * Cc_as / (3.0 * pi * mu_g)


# FUNCTION TO OBTAIN THE MOBILITY DIAMETER, Dm, from solving an equation
def SOLVE_Dm(Agg_f_agg):
    tol = 10e-4
    Dm_calc_0 = 1
    Cc_as = 1
    tol_calc = 100
    while tol_calc > tol:
        Dm_calc_1 = GET_Dm(Cc_as, Agg_f_agg)
        Cc_as = GET_Cc(Dm_calc_1 / 2.0)
        tol_calc = abs(Dm_calc_1 - Dm_calc_0)
        Dm_calc_0 = Dm_calc_1
    return Dm_calc_1


# ITERATIVE METHOD TO OBTAIN THE MOBILITY DIMATER, Dm
Aggregates["Dmm"] = Aggregates["f_agg"].progress_apply(SOLVE_Dm)


Aggregates

In [ ]:
ii = 0
for i in range(0, len(Aggregates.Dm)):
    for j in range(0, len(Aggregates.Dm[i])):
        Aggregates.loc[(i, j), "Dm"] = Dmm[ii]
        ii += 1

In [ ]:
Aggregates["DgOverDm"] = Aggregates["Dg"] / Aggregates["Dm"]

In [ ]:
# for i in range(0,len(Aggregates.f_agg[:,0])):
#    print("progress: " + str(i) + "/" + str(len(Aggregates.f_agg[:,0])))
#    for j in range(0,len(Aggregates.f_agg[:,1])):
#        print("prog: " + str(j) + "/" + str(len(Aggregates.f_agg[:,1])))
#        Dmm_sel.loc[(i,j),:] = Dmm[i,j]

In [ ]:
# THIRD: PLOT AND CALCULATE THE FRACTAL PARAMETERS
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn import linear_model

# LIMIT OF TIME: select aggregates up to a specific time
times = Aggregates.index.get_level_values(0)
tlimit = times.max() * 0.3
Selection = Aggregates.loc[tlimit:]

# LIMIT OF Np: select aggregates with more than 15 monomers
Selection = Selection[Selection.Np > 15]

# PLOT: Now, do the plot: log-log
fig, ax = plt.subplots()
ax.loglog(Selection["DgOverDp"], Selection["Np"], ".", label="Aggregates")
ax.set_xlabel("Dg/Dp", fontsize=9)
ax.set_ylabel("Mean Np", fontsize=9)

# fit the fractal law (not the simplest way to do it)
LOG_DgOverDp = np.log(Selection["DgOverDp"])[:, np.newaxis]
LOG_Np = np.log(Selection["Np"])

model = linear_model.LinearRegression()
model.fit(LOG_DgOverDp, LOG_Np)
print("Fractal Law: ", np.exp(model.intercept_), "x^", *model.coef_)

# Predict data of estimated models
X = np.linspace(np.log(Selection["DgOverDp"]).min(), np.log(Selection["DgOverDp"]).max())[
    :, np.newaxis
]
Y = model.predict(X)
X = np.exp(X)
Y = np.exp(Y)

plt.plot(X, Y, label="Fit")
fig.suptitle("Evolution of Np in function of Dg/Dp", fontsize=11)
plt.legend()
plt.show()

In [ ]:
# LIMIT ON TIME:

%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn import linear_model

times = Aggregates.index.get_level_values(0)

Df_it = np.empty(13)
kf_it = np.empty(13)
percentage = range(10, 75, 5)
itera = 0
for i in percentage:
    tlimit2 = times.max() * i / 100
    Selection2 = Aggregates.loc[tlimit2:]

    TIME_LOG_DgOverDp = np.log(Selection2["DgOverDp"])
    TIME_LOG_Np = np.log(Selection2["Np"])

    TIME_LOG_DgOverDp = TIME_LOG_DgOverDp.groupby(by="Time").agg((np.mean))[:, np.newaxis]
    TIME_LOG_Np = TIME_LOG_Np.groupby(by="Time").agg((np.mean))[:, np.newaxis]

    model = linear_model.LinearRegression()
    model.fit(TIME_LOG_DgOverDp, TIME_LOG_Np)

    Df_it[itera] = float(*model.coef_)
    kf_it[itera] = np.exp(model.intercept_)

    itera += 1

fig, ax = plt.subplots()
ax.plot(percentage, Df_it, ".", label="Df")
ax.set_ylabel("Df", fontsize=9)
ax.set_xlabel("% time (from % to end)", fontsize=9)

plt.plot(percentage, kf_it, ".", label="kf")

In [ ]:
# LIMIT ON Np
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn import linear_model

Selection2 = Aggregates[Aggregates.Np > 0]

TIME_LOG_DgOverDp = np.log(Selection2["DgOverDp"])
TIME_LOG_Np = np.log(Selection2["Np"])

TIME_LOG_DgOverDp = TIME_LOG_DgOverDp.groupby(by="Time").agg((np.mean))[:, np.newaxis]
TIME_LOG_Np = TIME_LOG_Np.groupby(by="Time").agg((np.mean))[:, np.newaxis]

TIME_LOG_DgOverDp = TIME_LOG_DgOverDp.groupby(by="Np").agg((np.mean))[:, np.newaxis]
TIME_LOG_Np = TIME_LOG_Np.groupby(by="Np").agg((np.mean))[:, np.newaxis]

In [ ]:
fig, ax = plt.subplots()
ax.plot(TIME_LOG_DgOverDp, TIME_LOG_Np, ".", label="Aggregates")
ax.set_xlabel("Mean log Dg/Dp", fontsize=9)
ax.set_ylabel("Mean log Np", fontsize=9)

model = linear_model.LinearRegression()
model.fit(TIME_LOG_DgOverDp, TIME_LOG_Np)

print("Fractal Law: ", np.exp(model.intercept_), "x^", *model.coef_)

# Predict data of estimated models
X = np.linspace(TIME_LOG_DgOverDp.min(), TIME_LOG_DgOverDp.max())
Y = model.intercept_ + float(*model.coef_) * X

plt.plot(X, Y, label="Fit")
fig.suptitle("Evolution of Np in function of Dg/Dp", fontsize=11)
plt.legend()
plt.show()

len(TIME_LOG_DgOverDp)

In [ ]:
# Compute the mean for each time step
time_agg = Aggregates.groupby(by="Time").agg((np.mean))

# Selection = Aggregates
# Now, do the plot: log-log
fig, ax = plt.subplots()
ax.loglog(time_agg["DgOverDp"], time_agg["Np"], ".", label="Aggregates")
ax.set_xlabel("Mean Dg/Dp", fontsize=9)
ax.set_ylabel("Mean Np", fontsize=9)

# ignore the too small aggregates
# subset = Aggregates[Aggregates.Np > 2]

# Fit the fractal law (not the simplest way to do it)
# Fit with a lower bound limit

time_agg2 = time_agg[time_agg.Np > 3]
model = linear_model.LinearRegression()
model.fit(np.log(time_agg2["DgOverDp"])[:, np.newaxis], np.log(time_agg2["Np"]))

print("Fractal Law: ", np.exp(model.intercept_), "x^", *model.coef_)

# Predict data of estimated models
X = np.linspace(np.log(time_agg2["DgOverDp"]).min(), np.log(time_agg2["DgOverDp"]).max())[
    :, np.newaxis
]
Y = model.predict(X)
X = np.exp(X)
Y = np.exp(Y)

plt.plot(X, Y, label="Fit")
fig.suptitle("Evolution of Np in function of Dg/Dp", fontsize=11)
plt.legend()
plt.show()

In [ ]:
%matplotlib notebook
from DLCAtools import view_agg

t, label = Selection["DgOverDp"].idxmax()
Agg_max_Rg = Selection.loc[t, label]

Agg = Agg_max_Rg
view_agg(Agg, Spheres)
Agg

In [ ]:
# Compute the mean for each time step
%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.loglog(time_agg["Np"])
ax.set_xlabel("Time (s)", fontsize=9)
ax.set_ylabel("Mean Np", fontsize=9)
fig.suptitle("Time evolution of Np", fontsize=11)
plt.show()

time_agg_s = Aggregates.groupby(by="Time").agg((np.std))

%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.loglog(time_agg_s["Rg"])
ax.set_xlabel("Time (s)", fontsize=9)
ax.set_ylabel("Std Rg", fontsize=9)
fig.suptitle("Time evolution of std(Rg)", fontsize=11)
plt.show()

In [ ]:
time_size = Aggregates.groupby(by="Time").agg((np.size))
time_size_Nc = time_size.Np

%matplotlib notebook
import matplotlib.pyplot as plt

T = 1700
L = 2.8281e-06

k_B = 1.38066e-23
mu_g = 18.203e-6 * (293.15 + 110) / (T + 110) * (T / 293.15) ** (1.5)

time_size_Nc = time_size_Nc / L**3

K0 = k_B * T / (3.0 * mu_g)
N0 = time_size_Nc[0]

# Nt = N0./(1+N0*K0*time_size[:,0])

time_size_Np.to_csv("time_size_Np.csv", header=True)

fig, ax = plt.subplots()
ax.loglog(time_size_Np)
ax.set_xlabel("Time (s)", fontsize=9)
ax.set_ylabel("Nc(t)", fontsize=9)
fig.suptitle("Time evolution of the Number of clusters", fontsize=11)
plt.show()